In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import librosa
import librosa.display
import re
from sklearn.model_selection import train_test_split
import math
import keras
from keras import backend as K

In [2]:
ds_dir = '/arch/mnist_data'
filenames = []

for root, dirs, files in os.walk(ds_dir):
    for file in files:
        filenames.append(os.path.join(root,file))
filenames = [re.sub(r'(\\)', '/', name) for name in filenames[1:]]

samples_number = len(filenames)
print('Total number of samples:', samples_number)

Total number of samples: 30000


In [3]:
labels = np.array(["0","1","2","3","4","5","6","7","8","9"])
train_files,test_files = train_test_split(filenames, test_size=0.3, shuffle=True, random_state=1)
test_files,val_files = train_test_split(test_files, test_size=0.33, shuffle=True, random_state=1)

print('Training data size', len(train_files))
print('Test data size', len(test_files))
print('Validation data size', len(val_files))

Training data size 21000
Test data size 6030
Validation data size 2970


In [4]:
def get_label(file_path):
    label = re.findall(r'/([0-9])_', file_path)[0]
    index = np.argmax(label == labels)
    label_id = np.zeros(len(labels))
    label_id[index] = 1
    return label_id

In [5]:
def get_ms(file_path, input_shape):
    y, sr = librosa.load(file_path)
    ms = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)

    ms = np.resize(ms, input_shape)
    ms = np.log(ms + 1e-9)
    ms = librosa.util.normalize(ms)
    
    ms = np.repeat(ms[..., np.newaxis], 3, -1)
    return ms

In [6]:
def preprocess_dataset(files, input_shape):
    output_labels = np.array([get_label(file) for file in files])
    output_ms = np.array([get_ms(file, input_shape) for file in files])
    return output_ms, output_labels

In [7]:
batch_size = 64

epoch=10
fine_tune_epochs = 15
total_epochs =  epoch + fine_tune_epochs

In [8]:
tf.compat.v1.enable_eager_execution()

In [9]:
input_shape = (128,44)
train_ms,train_labels = preprocess_dataset(train_files, input_shape)
test_ms,test_labels = preprocess_dataset(test_files, input_shape)
val_ms,val_labels = preprocess_dataset(val_files, input_shape)

train_dataset = tf.data.Dataset.from_tensor_slices((train_ms, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_ms, test_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_ms, val_labels))

train_batches = train_dataset.batch(batch_size)
test_batches = test_dataset.batch(batch_size)
val_batches = val_dataset.batch(batch_size)

In [10]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [11]:
import sklearn

def metric_calc(model):
    y_true=test_labels
    y_pred = model.predict(test_batches, verbose=2)

    y_pred = np.array([np.argmax(i) for i in y_pred])
    y_true = np.array([np.argmax(i) for i in y_true])

    accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
    precision = sklearn.metrics.precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = sklearn.metrics.recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = sklearn.metrics.f1_score(y_true, y_pred, average='weighted', zero_division=0)

    return accuracy, precision, recall, f1

# DenseNet121

In [14]:
model_densenet121 = tf.keras.applications.densenet.DenseNet121(input_shape=input_shape+(3,), include_top=False, 
                                                               weights='imagenet')
model_densenet121.trainable = False

densenet121 = keras.Sequential([
    model_densenet121,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                
])

densenet121.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_densenet121 = densenet121.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches,
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 176s 519ms/step - loss: 1.3798 - accuracy: 0.5310 - f1_score: 0.4205 - precision: 0.2881 - recall: 0.7873 - val_loss: 1.4040 - val_accuracy: 0.5135 - val_f1_score: 0.4190 - val_precision: 0.2789 - val_recall: 0.8433
Epoch 2/10
329/329 [==============================] - 169s 515ms/step - loss: 1.0206 - accuracy: 0.6481 - f1_score: 0.4497 - precision: 0.2992 - recall: 0.9066 - val_loss: 1.1592 - val_accuracy: 0.5936 - val_f1_score: 0.4373 - val_precision: 0.2893 - val_recall: 0.8965
Epoch 3/10
329/329 [==============================] - 169s 515ms/step - loss: 0.9304 - accuracy: 0.6819 - f1_score: 0.4518 - precision: 0.2989 - recall: 0.9266 - val_loss: 0.9828 - val_accuracy: 0.6559 - val_f1_score: 0.4471 - val_precision: 0.2957 - val_recall: 0.9167
Epoch 4/10
329/329 [==============================] - 169s 515ms/step - loss: 0.8741 - accuracy: 0.7021 - f1_score: 0.4536 - precision: 0.2993 - recall: 0.9375 - val_loss: 0.9149 - val_accur

In [15]:
model_densenet121.trainable = True
fine_tune_at = int(len(model_densenet121.layers)*0.7)
for layer in model_densenet121.layers[:fine_tune_at]:
    layer.trainble = False
    
densenet121.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_densenet121_fine = densenet121.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_densenet121.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 797s 2s/step - loss: 0.8633 - accuracy: 0.7393 - f1_score: 0.3901 - precision: 0.2457 - recall: 0.9540 - val_loss: 0.8593 - val_accuracy: 0.7519 - val_f1_score: 0.3916 - val_precision: 0.2450 - val_recall: 0.9759
Epoch 11/25
329/329 [==============================] - 789s 2s/step - loss: 0.2666 - accuracy: 0.9047 - f1_score: 0.4060 - precision: 0.2550 - recall: 0.9974 - val_loss: 0.3166 - val_accuracy: 0.8939 - val_f1_score: 0.4020 - val_precision: 0.2518 - val_recall: 0.9970
Epoch 12/25
329/329 [==============================] - 787s 2s/step - loss: 0.1411 - accuracy: 0.9516 - f1_score: 0.4090 - precision: 0.2572 - recall: 0.9992 - val_loss: 0.1535 - val_accuracy: 0.9495 - val_f1_score: 0.4116 - val_precision: 0.2593 - val_recall: 0.9987
Epoch 13/25
329/329 [==============================] - 791s 2s/step - loss: 0.0863 - accuracy: 0.9701 - f1_score: 0.4125 - precision: 0.2599 - recall: 0.9999 - val_loss: 0.1143 - val_accuracy: 0.9

In [16]:
metric_calc(densenet121)

95/95 - 71s - 71s/epoch - 750ms/step


(0.9800995024875622,
 0.9801142486092799,
 0.9800995024875622,
 0.9800800624688556)

In [17]:
test_loss, test_acc, test_f1, test_prec, test_rec = densenet121.evaluate(test_batches, verbose=2)

95/95 - 64s - loss: 0.0853 - accuracy: 0.9801 - f1_score: 0.4243 - precision: 0.2693 - recall: 0.9997 - 64s/epoch - 676ms/step


In [18]:
densenet121.save("models/densenet121_weights")

INFO:tensorflow:Assets written to: models/densenet121_weights\assets


INFO:tensorflow:Assets written to: models/densenet121_weights\assets


# DenseNet169

In [19]:
model_densenet169 = tf.keras.applications.densenet.DenseNet169(input_shape=input_shape+(3,), include_top=False, 
                                                               weights='imagenet')
model_densenet169.trainable = False

densenet169 = keras.Sequential([
    model_densenet169,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

densenet169.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_densenet169 = densenet169.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 222s 650ms/step - loss: 1.3188 - accuracy: 0.5439 - f1_score: 0.3874 - precision: 0.2519 - recall: 0.8470 - val_loss: 1.1399 - val_accuracy: 0.5990 - val_f1_score: 0.3912 - val_precision: 0.2481 - val_recall: 0.9259
Epoch 2/10
329/329 [==============================] - 246s 749ms/step - loss: 0.9373 - accuracy: 0.6795 - f1_score: 0.4123 - precision: 0.2637 - recall: 0.9460 - val_loss: 0.9948 - val_accuracy: 0.6519 - val_f1_score: 0.3970 - val_precision: 0.2514 - val_recall: 0.9438
Epoch 3/10
329/329 [==============================] - 324s 985ms/step - loss: 0.8490 - accuracy: 0.7111 - f1_score: 0.4112 - precision: 0.2619 - recall: 0.9579 - val_loss: 0.8917 - val_accuracy: 0.6875 - val_f1_score: 0.4071 - val_precision: 0.2588 - val_recall: 0.9541
Epoch 4/10
329/329 [==============================] - 327s 993ms/step - loss: 0.7929 - accuracy: 0.7310 - f1_score: 0.4111 - precision: 0.2613 - recall: 0.9644 - val_loss: 0.8759 - val_accur

In [20]:
model_densenet169.trainable = True
fine_tune_at = int(len(model_densenet169.layers)*0.7)
for layer in model_densenet169.layers[:fine_tune_at]:
    layer.trainble = False
    
densenet169.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_densenet169_fine = densenet169.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_densenet169.epoch[-1],
                    validation_data=val_batches,
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 1339s 4s/step - loss: 1.2621 - accuracy: 0.6544 - f1_score: 0.3864 - precision: 0.2441 - recall: 0.9321 - val_loss: 222833.6250 - val_accuracy: 0.1040 - val_f1_score: 0.1735 - val_precision: 0.1012 - val_recall: 0.6073
Epoch 11/25
329/329 [==============================] - 1531s 5s/step - loss: 0.7691 - accuracy: 0.7851 - f1_score: 0.3922 - precision: 0.2449 - recall: 0.9852 - val_loss: 0.9553 - val_accuracy: 0.7017 - val_f1_score: 0.3875 - val_precision: 0.2411 - val_recall: 0.9864
Epoch 12/25
329/329 [==============================] - 1491s 5s/step - loss: 0.3748 - accuracy: 0.8824 - f1_score: 0.4018 - precision: 0.2517 - recall: 0.9967 - val_loss: 0.9323 - val_accuracy: 0.8397 - val_f1_score: 0.3950 - val_precision: 0.2465 - val_recall: 0.9937
Epoch 13/25
329/329 [==============================] - 1526s 5s/step - loss: 0.2156 - accuracy: 0.9305 - f1_score: 0.4012 - precision: 0.2511 - recall: 0.9990 - val_loss: 0.1926 - val_accu

In [21]:
metric_calc(densenet169)

95/95 - 107s - 107s/epoch - 1s/step


(0.9733001658374792,
 0.9732944548117296,
 0.9733001658374792,
 0.9732878687409516)

In [22]:
test_loss, test_acc, test_f1, test_prec, test_rec = densenet169.evaluate(test_batches, verbose=2)

95/95 - 99s - loss: 0.1249 - accuracy: 0.9733 - f1_score: 0.4019 - precision: 0.2515 - recall: 0.9998 - 99s/epoch - 1s/step


In [23]:
densenet169.save("models/densenet169_weights")

INFO:tensorflow:Assets written to: models/densenet169_weights\assets


INFO:tensorflow:Assets written to: models/densenet169_weights\assets


# DenseNet201

In [24]:
model_densenet201 = tf.keras.applications.densenet.DenseNet201(input_shape=input_shape+(3,), include_top=False, 
                                                               weights='imagenet')
model_densenet201.trainable = False

densenet201 = keras.Sequential([
    model_densenet201,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

densenet201.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_densenet201 = densenet201.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches,                                     
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 382s 1s/step - loss: 1.2442 - accuracy: 0.5761 - f1_score: 0.3697 - precision: 0.2334 - recall: 0.8916 - val_loss: 0.9608 - val_accuracy: 0.6825 - val_f1_score: 0.3933 - val_precision: 0.2479 - val_recall: 0.9525
Epoch 2/10
329/329 [==============================] - 369s 1s/step - loss: 0.8563 - accuracy: 0.7107 - f1_score: 0.3923 - precision: 0.2463 - recall: 0.9654 - val_loss: 0.8324 - val_accuracy: 0.7253 - val_f1_score: 0.3913 - val_precision: 0.2452 - val_recall: 0.9684
Epoch 3/10
329/329 [==============================] - 368s 1s/step - loss: 0.7599 - accuracy: 0.7432 - f1_score: 0.3938 - precision: 0.2468 - recall: 0.9748 - val_loss: 0.7705 - val_accuracy: 0.7475 - val_f1_score: 0.3904 - val_precision: 0.2442 - val_recall: 0.9734
Epoch 4/10
329/329 [==============================] - 368s 1s/step - loss: 0.7053 - accuracy: 0.7615 - f1_score: 0.3933 - precision: 0.2462 - recall: 0.9779 - val_loss: 0.7495 - val_accuracy: 0.7505 

In [25]:
model_densenet201.trainable = True
fine_tune_at = int(len(model_densenet201.layers)*0.7)
for layer in model_densenet201.layers[:fine_tune_at]:
    layer.trainble = False
    
densenet201.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_densenet201_fine = densenet201.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_densenet201.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 1865s 6s/step - loss: 0.6988 - accuracy: 0.7940 - f1_score: 0.3836 - precision: 0.2387 - recall: 0.9781 - val_loss: 0.8185 - val_accuracy: 0.7623 - val_f1_score: 0.3607 - val_precision: 0.2205 - val_recall: 0.9917
Epoch 11/25
329/329 [==============================] - 1824s 6s/step - loss: 0.1823 - accuracy: 0.9366 - f1_score: 0.4108 - precision: 0.2586 - recall: 0.9991 - val_loss: 0.2034 - val_accuracy: 0.9313 - val_f1_score: 0.4093 - val_precision: 0.2574 - val_recall: 0.9993
Epoch 12/25
329/329 [==============================] - 2060s 6s/step - loss: 0.0925 - accuracy: 0.9677 - f1_score: 0.4096 - precision: 0.2576 - recall: 0.9998 - val_loss: 0.1537 - val_accuracy: 0.9495 - val_f1_score: 0.4092 - val_precision: 0.2573 - val_recall: 0.9993
Epoch 13/25
329/329 [==============================] - 1525s 5s/step - loss: 0.0513 - accuracy: 0.9830 - f1_score: 0.4118 - precision: 0.2594 - recall: 1.0000 - val_loss: 0.1680 - val_accuracy:

In [26]:
metric_calc(densenet201)

95/95 - 164s - 164s/epoch - 2s/step


(0.9837479270315092,
 0.9837436243558704,
 0.9837479270315092,
 0.9837319471256245)

In [27]:
test_loss, test_acc, test_f1, test_prec, test_rec = densenet201.evaluate(test_batches, verbose=2)

95/95 - 155s - loss: 0.0667 - accuracy: 0.9837 - f1_score: 0.4172 - precision: 0.2637 - recall: 0.9997 - 155s/epoch - 2s/step


In [28]:
densenet201.save("models/densenet201_weights")

INFO:tensorflow:Assets written to: models/densenet201_weights\assets


INFO:tensorflow:Assets written to: models/densenet201_weights\assets


# MobileNet

In [29]:
model_mobilenet = tf.keras.applications.mobilenet.MobileNet(input_shape=input_shape+(3,), include_top=False, 
                                               weights='imagenet')
model_mobilenet.trainable = False

mobilenet = keras.Sequential([
    model_mobilenet,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

mobilenet.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenet = mobilenet.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 68s 202ms/step - loss: 1.5784 - accuracy: 0.4619 - f1_score: 0.3500 - precision: 0.2269 - recall: 0.7794 - val_loss: 1.3567 - val_accuracy: 0.5438 - val_f1_score: 0.3709 - val_precision: 0.2349 - val_recall: 0.8825
Epoch 2/10
329/329 [==============================] - 62s 187ms/step - loss: 1.2782 - accuracy: 0.5725 - f1_score: 0.3794 - precision: 0.2403 - recall: 0.9016 - val_loss: 1.2385 - val_accuracy: 0.5865 - val_f1_score: 0.3767 - val_precision: 0.2378 - val_recall: 0.9074
Epoch 3/10
329/329 [==============================] - 92s 280ms/step - loss: 1.1925 - accuracy: 0.6035 - f1_score: 0.3818 - precision: 0.2410 - recall: 0.9193 - val_loss: 1.1829 - val_accuracy: 0.6084 - val_f1_score: 0.3792 - val_precision: 0.2391 - val_recall: 0.9161
Epoch 4/10
329/329 [==============================] - 89s 272ms/step - loss: 1.1448 - accuracy: 0.6220 - f1_score: 0.3826 - precision: 0.2411 - recall: 0.9277 - val_loss: 1.1524 - val_accuracy:

In [30]:
model_mobilenet.trainable = True
fine_tune_at = int(len(model_mobilenet.layers)*0.7)
for layer in model_mobilenet.layers[:fine_tune_at]:
    layer.trainble = False
    
mobilenet.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenet_fine = mobilenet.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_mobilenet.epoch[-1],
                    validation_data=val_batches,
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 454s 1s/step - loss: 0.7606 - accuracy: 0.8023 - f1_score: 0.3773 - precision: 0.2338 - recall: 0.9797 - val_loss: 0.2846 - val_accuracy: 0.8983 - val_f1_score: 0.4072 - val_precision: 0.2559 - val_recall: 0.9977
Epoch 11/25
329/329 [==============================] - 443s 1s/step - loss: 0.1413 - accuracy: 0.9501 - f1_score: 0.4034 - precision: 0.2528 - recall: 0.9987 - val_loss: 0.1939 - val_accuracy: 0.9350 - val_f1_score: 0.4051 - val_precision: 0.2541 - val_recall: 0.9990
Epoch 12/25
329/329 [==============================] - 443s 1s/step - loss: 0.0553 - accuracy: 0.9820 - f1_score: 0.4073 - precision: 0.2558 - recall: 0.9999 - val_loss: 0.1077 - val_accuracy: 0.9653 - val_f1_score: 0.4097 - val_precision: 0.2577 - val_recall: 0.9997
Epoch 13/25
329/329 [==============================] - 443s 1s/step - loss: 0.0223 - accuracy: 0.9933 - f1_score: 0.4086 - precision: 0.2568 - recall: 1.0000 - val_loss: 0.1068 - val_accuracy: 0.9

In [31]:
metric_calc(mobilenet)

95/95 - 23s - 23s/epoch - 247ms/step


(0.9772802653399668, 0.977389287279158, 0.9772802653399668, 0.9772595963702106)

In [32]:
test_loss, test_acc, test_f1, test_prec, test_rec = mobilenet.evaluate(test_batches, verbose=2)

95/95 - 22s - loss: 0.1028 - accuracy: 0.9773 - f1_score: 0.4100 - precision: 0.2579 - recall: 0.9995 - 22s/epoch - 229ms/step


In [33]:
mobilenet.save("models/mobilenet_weights")

INFO:tensorflow:Assets written to: models/mobilenet_weights\assets


INFO:tensorflow:Assets written to: models/mobilenet_weights\assets


# MobileNetV2

In [12]:
model_mobilenetv2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=input_shape+(3,), include_top=False, 
                                               weights='imagenet', alpha=1.0)
model_mobilenetv2.trainable = False

mobilenetv2 = keras.Sequential([
    model_mobilenetv2,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

mobilenetv2.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenetv2 = mobilenetv2.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 85s 249ms/step - loss: 1.5312 - accuracy: 0.4722 - f1_score: 0.3249 - precision: 0.2008 - recall: 0.8524 - val_loss: 1.3027 - val_accuracy: 0.5539 - val_f1_score: 0.3468 - val_precision: 0.2133 - val_recall: 0.9274
Epoch 2/10
329/329 [==============================] - 85s 258ms/step - loss: 1.1871 - accuracy: 0.5984 - f1_score: 0.3498 - precision: 0.2152 - recall: 0.9348 - val_loss: 1.1748 - val_accuracy: 0.5970 - val_f1_score: 0.3527 - val_precision: 0.2168 - val_recall: 0.9450
Epoch 3/10
329/329 [==============================] - 85s 258ms/step - loss: 1.0919 - accuracy: 0.6339 - f1_score: 0.3541 - precision: 0.2177 - recall: 0.9483 - val_loss: 1.1129 - val_accuracy: 0.6226 - val_f1_score: 0.3547 - val_precision: 0.2181 - val_recall: 0.9503
Epoch 4/10
329/329 [==============================] - 85s 260ms/step - loss: 1.0379 - accuracy: 0.6529 - f1_score: 0.3557 - precision: 0.2186 - recall: 0.9544 - val_loss: 1.0744 - val_accuracy:

In [13]:
model_mobilenetv2.trainable = True
fine_tune_at = int(len(model_mobilenetv2.layers)*0.7)
for layer in model_mobilenetv2.layers[:fine_tune_at]:
    layer.trainble = False
    
mobilenetv2.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenetv2_fine = mobilenetv2.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_mobilenetv2.epoch[-1],
                    validation_data=val_batches,
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 329s 972ms/step - loss: 1.1461 - accuracy: 0.6142 - f1_score: 0.3878 - precision: 0.2554 - recall: 0.8407 - val_loss: 7.9879 - val_accuracy: 0.1387 - val_f1_score: 0.1871 - val_precision: 0.1140 - val_recall: 0.5218
Epoch 11/25
329/329 [==============================] - 327s 995ms/step - loss: 0.2842 - accuracy: 0.9017 - f1_score: 0.4504 - precision: 0.2911 - recall: 0.9965 - val_loss: 5.7285 - val_accuracy: 0.2232 - val_f1_score: 0.2349 - val_precision: 0.1458 - val_recall: 0.6044
Epoch 12/25
329/329 [==============================] - 328s 998ms/step - loss: 0.1369 - accuracy: 0.9521 - f1_score: 0.4459 - precision: 0.2870 - recall: 0.9994 - val_loss: 5.9491 - val_accuracy: 0.2027 - val_f1_score: 0.2977 - val_precision: 0.1855 - val_recall: 0.7531
Epoch 13/25
329/329 [==============================] - 327s 995ms/step - loss: 0.0801 - accuracy: 0.9717 - f1_score: 0.4488 - precision: 0.2894 - recall: 0.9999 - val_loss: 4.9517 - val_a

In [14]:
metric_calc(mobilenetv2)

95/95 - 24s - 24s/epoch - 257ms/step


(0.9739635157545605,
 0.9741467723333136,
 0.9739635157545605,
 0.9739541479157715)

In [15]:
test_loss, test_acc, test_f1, test_prec, test_rec = mobilenetv2.evaluate(test_batches, verbose=2)

95/95 - 21s - loss: 0.1386 - accuracy: 0.9740 - f1_score: 0.4516 - precision: 0.2918 - recall: 0.9993 - 21s/epoch - 221ms/step


In [16]:
mobilenetv2.save("models/mobilenetv2_weights")

INFO:tensorflow:Assets written to: models/mobilenetv2_weights\assets


INFO:tensorflow:Assets written to: models/mobilenetv2_weights\assets


# MobileNetV3Large

In [12]:
model_mobilenetv3large = tf.keras.applications.MobileNetV3Large(input_shape=input_shape+(3,), include_top=False, 
                                               weights='imagenet', include_preprocessing=False, alpha=1.0)
model_mobilenetv3large.trainable = False

mobilenetv3large = keras.Sequential([
    model_mobilenetv3large,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                 
])

mobilenetv3large.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenetv3large = mobilenetv3large.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches,
                    verbose=1,
                    shuffle=True)

D:\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
329/329 [==============================] - 54s 152ms/step - loss: 1.6782 - accuracy: 0.4201 - f1_score: 0.3726 - precision: 0.2606 - recall: 0.6618 - val_loss: 1.4808 - val_accuracy: 0.4865 - val_f1_score: 0.4044 - val_precision: 0.2756 - val_recall: 0.7610
Epoch 2/10
329/329 [==============================] - 49s 150ms/step - loss: 1.3364 - accuracy: 0.5467 - f1_score: 0.4229 - precision: 0.2858 - recall: 0.8151 - val_loss: 1.3211 - val_accuracy: 0.5451 - val_f1_score: 0.4163 - val_precision: 0.2778 - val_recall: 0.8322
Epoch 3/10
329/329 [==============================] - 49s 149ms/step - loss: 1.2451 - accuracy: 0.5786 - f1_score: 0.4275 - precision: 0.2861 - recall: 0.8469 - val_loss: 1.2523 - val_accuracy: 0.5673 - val_f1_score: 0.4194 - val_precision: 0.2784 - val_recall: 0.8516
Epoch 4/10
329/329 [==============================] - 49s 149ms/step - loss: 1.1890 - accuracy: 0.6007 - f1_score: 0.4313 - precision: 0.2876 - recall: 0.8640 - val_loss: 1.2083 - val_accuracy:

In [13]:
model_mobilenetv3large.trainable = True
fine_tune_at = int(len(model_mobilenetv3large.layers)*0.7)
for layer in model_mobilenetv3large.layers[:fine_tune_at]:
    layer.trainble = False

mobilenetv3large.compile(optimizer=keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenetv3large_fine = mobilenetv3large.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_mobilenetv3large.epoch[-1],
                    validation_data=val_batches,
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 193s 568ms/step - loss: 0.5577 - accuracy: 0.8205 - f1_score: 0.3822 - precision: 0.2375 - recall: 0.9793 - val_loss: 16.5784 - val_accuracy: 0.1633 - val_f1_score: 0.2004 - val_precision: 0.1204 - val_recall: 0.5987
Epoch 11/25
329/329 [==============================] - 186s 565ms/step - loss: 0.1113 - accuracy: 0.9620 - f1_score: 0.3974 - precision: 0.2480 - recall: 0.9996 - val_loss: 4.8150 - val_accuracy: 0.2758 - val_f1_score: 0.2426 - val_precision: 0.1452 - val_recall: 0.7359
Epoch 12/25
329/329 [==============================] - 186s 564ms/step - loss: 0.0310 - accuracy: 0.9916 - f1_score: 0.3990 - precision: 0.2493 - recall: 1.0000 - val_loss: 2.2299 - val_accuracy: 0.4791 - val_f1_score: 0.3399 - val_precision: 0.2080 - val_recall: 0.9292
Epoch 13/25
329/329 [==============================] - 186s 564ms/step - loss: 0.0088 - accuracy: 0.9986 - f1_score: 0.4008 - precision: 0.2506 - recall: 1.0000 - val_loss: 1.0228 - val_

In [14]:
metric_calc(mobilenetv3large)

95/95 - 13s - 13s/epoch - 141ms/step


(0.966832504145937, 0.9668456535032656, 0.966832504145937, 0.9667717093801858)

In [15]:
test_loss, test_acc, test_f1, test_prec, test_rec = mobilenetv3large.evaluate(test_batches, verbose=2)

95/95 - 12s - loss: 0.1354 - accuracy: 0.9668 - f1_score: 0.4060 - precision: 0.2548 - recall: 0.9993 - 12s/epoch - 128ms/step


In [16]:
mobilenetv3large.save("models/mobilenetv3large_weights")

INFO:tensorflow:Assets written to: models/mobilenetv3large_weights\assets


INFO:tensorflow:Assets written to: models/mobilenetv3large_weights\assets


# MobileNetV3Small

In [17]:
model_mobilenetv3small = tf.keras.applications.MobileNetV3Small(input_shape=input_shape+(3,), include_top=False, 
                                               weights='imagenet', include_preprocessing=False, alpha=1.0)
model_mobilenetv3small.trainable = False

mobilenetv3small = keras.Sequential([
    model_mobilenetv3small,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                   
])

mobilenetv3small.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenetv3small = mobilenetv3small.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches,
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 44s 119ms/step - loss: 1.7808 - accuracy: 0.3858 - f1_score: 0.3062 - precision: 0.1909 - recall: 0.7747 - val_loss: 1.5626 - val_accuracy: 0.4448 - val_f1_score: 0.3296 - val_precision: 0.2032 - val_recall: 0.8732
Epoch 2/10
329/329 [==============================] - 37s 113ms/step - loss: 1.4464 - accuracy: 0.5174 - f1_score: 0.3383 - precision: 0.2084 - recall: 0.8983 - val_loss: 1.3767 - val_accuracy: 0.5434 - val_f1_score: 0.3422 - val_precision: 0.2102 - val_recall: 0.9206
Epoch 3/10
329/329 [==============================] - 37s 113ms/step - loss: 1.3348 - accuracy: 0.5601 - f1_score: 0.3438 - precision: 0.2113 - recall: 0.9230 - val_loss: 1.2996 - val_accuracy: 0.5741 - val_f1_score: 0.3457 - val_precision: 0.2122 - val_recall: 0.9332
Epoch 4/10
329/329 [==============================] - 37s 112ms/step - loss: 1.2717 - accuracy: 0.5827 - f1_score: 0.3465 - precision: 0.2128 - recall: 0.9343 - val_loss: 1.2518 - val_accuracy:

In [18]:
model_mobilenetv3small.trainable = True
fine_tune_at = int(len(model_mobilenetv3small.layers)*0.7)
for layer in model_mobilenetv3small.layers[:fine_tune_at]:
    layer.trainble = False

mobilenetv3small.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_mobilenetv3small_fine = mobilenetv3small.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_mobilenetv3small.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 136s 376ms/step - loss: 0.6765 - accuracy: 0.7708 - f1_score: 0.3675 - precision: 0.2267 - recall: 0.9711 - val_loss: 2.7791 - val_accuracy: 0.2939 - val_f1_score: 0.2466 - val_precision: 0.1498 - val_recall: 0.6979
Epoch 11/25
329/329 [==============================] - 121s 369ms/step - loss: 0.1951 - accuracy: 0.9316 - f1_score: 0.3780 - precision: 0.2331 - recall: 0.9987 - val_loss: 2.2002 - val_accuracy: 0.3680 - val_f1_score: 0.3040 - val_precision: 0.1861 - val_recall: 0.8300
Epoch 12/25
329/329 [==============================] - 121s 368ms/step - loss: 0.0940 - accuracy: 0.9718 - f1_score: 0.3795 - precision: 0.2342 - recall: 0.9997 - val_loss: 2.0366 - val_accuracy: 0.4182 - val_f1_score: 0.3391 - val_precision: 0.2087 - val_recall: 0.9051
Epoch 13/25
329/329 [==============================] - 121s 368ms/step - loss: 0.0449 - accuracy: 0.9892 - f1_score: 0.3803 - precision: 0.2348 - recall: 1.0000 - val_loss: 1.3039 - val_a

In [19]:
metric_calc(mobilenetv3small)

95/95 - 11s - 11s/epoch - 118ms/step


(0.9464344941956883, 0.946906624899922, 0.9464344941956883, 0.9462917351515817)

In [20]:
test_loss, test_acc, test_f1, test_prec, test_rec = mobilenetv3small.evaluate(test_batches, verbose=2)

95/95 - 9s - loss: 0.2329 - accuracy: 0.9464 - f1_score: 0.3842 - precision: 0.2379 - recall: 0.9980 - 9s/epoch - 94ms/step


In [21]:
mobilenetv3small.save("models/mobilenetv3small_weights")

INFO:tensorflow:Assets written to: models/mobilenetv3small_weights\assets


INFO:tensorflow:Assets written to: models/mobilenetv3small_weights\assets


# RegNetX002

In [22]:
model_regnetx002 = tf.keras.applications.regnet.RegNetX002(input_shape=input_shape+(3,), include_top=False, 
                                               include_preprocessing=False, weights='imagenet')
model_regnetx002.trainable = False

regnetx002 = keras.Sequential([
    model_regnetx002,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

regnetx002.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnetx002 = regnetx002.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 47s 133ms/step - loss: 7.3072 - accuracy: 0.2074 - f1_score: 0.2171 - precision: 0.1303 - recall: 0.6626 - val_loss: 5.8811 - val_accuracy: 0.1684 - val_f1_score: 0.2376 - val_precision: 0.1417 - val_recall: 0.7349
Epoch 2/10
329/329 [==============================] - 42s 128ms/step - loss: 2.6577 - accuracy: 0.3346 - f1_score: 0.2892 - precision: 0.1794 - recall: 0.7511 - val_loss: 2.4279 - val_accuracy: 0.2973 - val_f1_score: 0.2980 - val_precision: 0.1895 - val_recall: 0.6987
Epoch 3/10
329/329 [==============================] - 37s 114ms/step - loss: 1.8723 - accuracy: 0.4031 - f1_score: 0.3243 - precision: 0.2046 - recall: 0.7850 - val_loss: 2.0161 - val_accuracy: 0.3593 - val_f1_score: 0.3183 - val_precision: 0.2025 - val_recall: 0.7438
Epoch 4/10
329/329 [==============================] - 38s 114ms/step - loss: 1.6799 - accuracy: 0.4354 - f1_score: 0.3403 - precision: 0.2158 - recall: 0.8061 - val_loss: 1.9735 - val_accuracy:

In [23]:
model_regnetx002.trainable = True
fine_tune_at = int(len(model_regnetx002.layers)*0.7)
for layer in model_regnetx002.layers[:fine_tune_at]:
    layer.trainble = False
    
regnetx002.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnetx002_fine = regnetx002.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_regnetx002.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 2075s 6s/step - loss: 0.9228 - accuracy: 0.7117 - f1_score: 0.3791 - precision: 0.2368 - recall: 0.9502 - val_loss: 0.4346 - val_accuracy: 0.8411 - val_f1_score: 0.3949 - val_precision: 0.2464 - val_recall: 0.9947
Epoch 11/25
329/329 [==============================] - 2544s 8s/step - loss: 0.2874 - accuracy: 0.8991 - f1_score: 0.4083 - precision: 0.2568 - recall: 0.9966 - val_loss: 0.2751 - val_accuracy: 0.9104 - val_f1_score: 0.4014 - val_precision: 0.2513 - val_recall: 0.9970
Epoch 12/25
329/329 [==============================] - 2553s 8s/step - loss: 0.1520 - accuracy: 0.9465 - f1_score: 0.4093 - precision: 0.2574 - recall: 0.9993 - val_loss: 0.2150 - val_accuracy: 0.9316 - val_f1_score: 0.4049 - val_precision: 0.2540 - val_recall: 0.9980
Epoch 13/25
329/329 [==============================] - 2562s 8s/step - loss: 0.0855 - accuracy: 0.9717 - f1_score: 0.4113 - precision: 0.2590 - recall: 0.9999 - val_loss: 0.1912 - val_accuracy:

In [25]:
metric_calc(regnetx002)

95/95 - 17s - 17s/epoch - 179ms/step


(0.9615257048092869,
 0.9616615091885057,
 0.9615257048092869,
 0.9615432887329942)

In [26]:
test_loss, test_acc, test_f1, test_prec, test_rec = regnetx002.evaluate(test_batches, verbose=2)

95/95 - 16s - loss: 0.1551 - accuracy: 0.9615 - f1_score: 0.4184 - precision: 0.2647 - recall: 0.9985 - 16s/epoch - 164ms/step


In [27]:
regnetx002.save("models/regnetx002_weights")

INFO:tensorflow:Assets written to: models/regnetx002_weights\assets


INFO:tensorflow:Assets written to: models/regnetx002_weights\assets


# RegNetX004

In [28]:
model_regnetx004 = tf.keras.applications.regnet.RegNetX004(input_shape=input_shape+(3,), include_top=False, 
                                               include_preprocessing=False, weights='imagenet')
model_regnetx004.trainable = False

regnetx004 = keras.Sequential([
    model_regnetx004,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

regnetx004.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnetx004 = regnetx004.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 72s 203ms/step - loss: 2.1188 - accuracy: 0.2641 - f1_score: 0.2824 - precision: 0.1911 - recall: 0.5494 - val_loss: 2.3427 - val_accuracy: 0.1990 - val_f1_score: 0.2837 - val_precision: 0.1868 - val_recall: 0.5894
Epoch 2/10
329/329 [==============================] - 60s 183ms/step - loss: 1.7822 - accuracy: 0.3680 - f1_score: 0.3398 - precision: 0.2256 - recall: 0.6922 - val_loss: 2.0886 - val_accuracy: 0.2458 - val_f1_score: 0.3212 - val_precision: 0.2105 - val_recall: 0.6778
Epoch 3/10
329/329 [==============================] - 60s 183ms/step - loss: 1.6436 - accuracy: 0.4174 - f1_score: 0.3602 - precision: 0.2382 - recall: 0.7411 - val_loss: 1.7479 - val_accuracy: 0.3431 - val_f1_score: 0.3488 - val_precision: 0.2290 - val_recall: 0.7319
Epoch 4/10
329/329 [==============================] - 72s 218ms/step - loss: 1.5591 - accuracy: 0.4509 - f1_score: 0.3724 - precision: 0.2455 - recall: 0.7740 - val_loss: 1.6043 - val_accuracy:

In [29]:
model_regnetx004.trainable = True
fine_tune_at = int(len(model_regnetx004.layers)*0.7)
for layer in model_regnetx004.layers[:fine_tune_at]:
    layer.trainble = False
    
regnetx004.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnetx004_fine = regnetx004.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_regnetx004.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 2474s 7s/step - loss: 0.9691 - accuracy: 0.6861 - f1_score: 0.3609 - precision: 0.2221 - recall: 0.9655 - val_loss: 0.4905 - val_accuracy: 0.8256 - val_f1_score: 0.3956 - val_precision: 0.2469 - val_recall: 0.9960
Epoch 11/25
329/329 [==============================] - 2470s 8s/step - loss: 0.3318 - accuracy: 0.8827 - f1_score: 0.3944 - precision: 0.2459 - recall: 0.9975 - val_loss: 0.2967 - val_accuracy: 0.8976 - val_f1_score: 0.3965 - val_precision: 0.2474 - val_recall: 0.9990
Epoch 12/25
329/329 [==============================] - 2481s 8s/step - loss: 0.1846 - accuracy: 0.9350 - f1_score: 0.4006 - precision: 0.2506 - recall: 0.9994 - val_loss: 0.2201 - val_accuracy: 0.9259 - val_f1_score: 0.3995 - val_precision: 0.2498 - val_recall: 0.9987
Epoch 13/25
329/329 [==============================] - 2486s 8s/step - loss: 0.1116 - accuracy: 0.9608 - f1_score: 0.4020 - precision: 0.2516 - recall: 0.9998 - val_loss: 0.2297 - val_accuracy:

In [30]:
metric_calc(regnetx004)

95/95 - 16s - 16s/epoch - 172ms/step


(0.9641791044776119, 0.9641392755341324, 0.9641791044776119, 0.964123742001244)

In [31]:
test_loss, test_acc, test_f1, test_prec, test_rec = regnetx004.evaluate(test_batches, verbose=2)

95/95 - 13s - loss: 0.1535 - accuracy: 0.9642 - f1_score: 0.4097 - precision: 0.2577 - recall: 0.9995 - 13s/epoch - 140ms/step


In [32]:
regnetx004.save("models/regnetx004_weights")

INFO:tensorflow:Assets written to: models/regnetx004_weights\assets


INFO:tensorflow:Assets written to: models/regnetx004_weights\assets


# RegNetX006

In [12]:
model_regnetx006 = tf.keras.applications.regnet.RegNetX006(input_shape=input_shape+(3,), include_top=False, 
                                               include_preprocessing=False, weights='imagenet')
model_regnetx006.trainable = False

regnetx006 = keras.Sequential([
    model_regnetx006,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

regnetx006.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnetx006 = regnetx006.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 75s 217ms/step - loss: 10.0199 - accuracy: 0.1722 - f1_score: 0.1983 - precision: 0.1166 - recall: 0.6729 - val_loss: 8.4986 - val_accuracy: 0.1559 - val_f1_score: 0.2205 - val_precision: 0.1321 - val_recall: 0.6661
Epoch 2/10
329/329 [==============================] - 71s 215ms/step - loss: 2.6874 - accuracy: 0.3153 - f1_score: 0.2638 - precision: 0.1582 - recall: 0.7976 - val_loss: 2.4879 - val_accuracy: 0.2471 - val_f1_score: 0.2631 - val_precision: 0.1591 - val_recall: 0.7594
Epoch 3/10
329/329 [==============================] - 71s 214ms/step - loss: 1.8625 - accuracy: 0.3879 - f1_score: 0.2866 - precision: 0.1725 - recall: 0.8506 - val_loss: 2.2196 - val_accuracy: 0.3636 - val_f1_score: 0.2553 - val_precision: 0.1544 - val_recall: 0.7367
Epoch 4/10
329/329 [==============================] - 71s 215ms/step - loss: 1.7573 - accuracy: 0.4134 - f1_score: 0.2935 - precision: 0.1767 - recall: 0.8674 - val_loss: 2.0551 - val_accuracy

In [13]:
model_regnetx006.trainable = True
fine_tune_at = int(len(model_regnetx006.layers)*0.7)
for layer in model_regnetx006.layers[:fine_tune_at]:
    layer.trainble = False
    
regnetx006.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnetx006_fine = regnetx006.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_regnetx006.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 1590s 5s/step - loss: 0.7808 - accuracy: 0.7467 - f1_score: 0.4011 - precision: 0.2536 - recall: 0.9590 - val_loss: 0.3376 - val_accuracy: 0.8785 - val_f1_score: 0.4211 - val_precision: 0.2670 - val_recall: 0.9960
Epoch 11/25
329/329 [==============================] - 1590s 5s/step - loss: 0.2345 - accuracy: 0.9176 - f1_score: 0.4253 - precision: 0.2703 - recall: 0.9979 - val_loss: 0.1974 - val_accuracy: 0.9337 - val_f1_score: 0.4208 - val_precision: 0.2666 - val_recall: 0.9983
Epoch 12/25
329/329 [==============================] - 1582s 5s/step - loss: 0.1138 - accuracy: 0.9603 - f1_score: 0.4272 - precision: 0.2717 - recall: 0.9996 - val_loss: 0.1578 - val_accuracy: 0.9458 - val_f1_score: 0.4308 - val_precision: 0.2747 - val_recall: 0.9983
Epoch 13/25
329/329 [==============================] - 1578s 5s/step - loss: 0.0599 - accuracy: 0.9801 - f1_score: 0.4276 - precision: 0.2720 - recall: 0.9998 - val_loss: 0.1331 - val_accuracy:

In [14]:
metric_calc(regnetx006)

95/95 - 19s - 19s/epoch - 203ms/step


(0.9703150912106135,
 0.9703389094199214,
 0.9703150912106135,
 0.9702828484728316)

In [15]:
test_loss, test_acc, test_f1, test_prec, test_rec = regnetx006.evaluate(test_batches, verbose=2)

95/95 - 15s - loss: 0.1161 - accuracy: 0.9703 - f1_score: 0.4361 - precision: 0.2790 - recall: 0.9990 - 15s/epoch - 161ms/step


In [16]:
regnetx006.save("models/regnetx006_weights")

INFO:tensorflow:Assets written to: models/regnetx006_weights\assets


INFO:tensorflow:Assets written to: models/regnetx006_weights\assets


# RegNetY002

In [19]:
model_regnety002 = tf.keras.applications.regnet.RegNetY002(input_shape=input_shape+(3,), include_top=False, 
                                               include_preprocessing=False, weights='imagenet')
model_regnety002.trainable = False

regnety002 = keras.Sequential([
    model_regnety002,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

regnety002.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnety002 = regnety002.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 45s 126ms/step - loss: 2.1973 - accuracy: 0.2028 - f1_score: 0.1655 - precision: 0.2107 - recall: 0.1607 - val_loss: 2.1019 - val_accuracy: 0.2620 - val_f1_score: 0.2822 - val_precision: 0.2464 - val_recall: 0.3329
Epoch 2/10
329/329 [==============================] - 38s 117ms/step - loss: 2.0542 - accuracy: 0.2952 - f1_score: 0.3084 - precision: 0.2425 - recall: 0.4302 - val_loss: 2.0215 - val_accuracy: 0.2916 - val_f1_score: 0.3228 - val_precision: 0.2382 - val_recall: 0.5037
Epoch 3/10
329/329 [==============================] - 39s 117ms/step - loss: 1.9884 - accuracy: 0.3308 - f1_score: 0.3302 - precision: 0.2388 - recall: 0.5379 - val_loss: 1.9745 - val_accuracy: 0.3155 - val_f1_score: 0.3321 - val_precision: 0.2341 - val_recall: 0.5739
Epoch 4/10
329/329 [==============================] - 39s 118ms/step - loss: 1.9465 - accuracy: 0.3501 - f1_score: 0.3372 - precision: 0.2362 - recall: 0.5913 - val_loss: 1.9415 - val_accuracy:

In [20]:
model_regnety002.trainable = True
fine_tune_at = int(len(model_regnety002.layers)*0.7)
for layer in model_regnety002.layers[:fine_tune_at]:
    layer.trainble = False
    
regnety002.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnety002_fine = regnety002.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_regnety002.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 1799s 5s/step - loss: 1.3935 - accuracy: 0.5120 - f1_score: 0.3409 - precision: 0.2181 - recall: 0.8268 - val_loss: 1.2636 - val_accuracy: 0.5380 - val_f1_score: 0.3504 - val_precision: 0.2155 - val_recall: 0.9377
Epoch 11/25
329/329 [==============================] - 1800s 5s/step - loss: 0.6894 - accuracy: 0.7591 - f1_score: 0.3527 - precision: 0.2148 - recall: 0.9858 - val_loss: 0.6059 - val_accuracy: 0.7889 - val_f1_score: 0.3534 - val_precision: 0.2151 - val_recall: 0.9900
Epoch 12/25
329/329 [==============================] - 1822s 6s/step - loss: 0.4760 - accuracy: 0.8333 - f1_score: 0.3553 - precision: 0.2163 - recall: 0.9936 - val_loss: 0.4667 - val_accuracy: 0.8370 - val_f1_score: 0.3610 - val_precision: 0.2207 - val_recall: 0.9930
Epoch 13/25
329/329 [==============================] - 1828s 6s/step - loss: 0.3561 - accuracy: 0.8789 - f1_score: 0.3588 - precision: 0.2188 - recall: 0.9967 - val_loss: 0.4168 - val_accuracy:

In [21]:
metric_calc(regnety002)

95/95 - 12s - 12s/epoch - 127ms/step


(0.896849087893864, 0.8995198741961693, 0.896849087893864, 0.8975075976650394)

In [22]:
test_loss, test_acc, test_f1, test_prec, test_rec = regnety002.evaluate(test_batches, verbose=2)

95/95 - 10s - loss: 0.3266 - accuracy: 0.8968 - f1_score: 0.3597 - precision: 0.2195 - recall: 0.9969 - 10s/epoch - 101ms/step


In [23]:
regnety002.save("models/regnety002_weights")

INFO:tensorflow:Assets written to: models/regnety002_weights\assets


INFO:tensorflow:Assets written to: models/regnety002_weights\assets


# RegNetY004

In [13]:
model_regnety004 = tf.keras.applications.regnet.RegNetY004(input_shape=input_shape+(3,), include_top=False, 
                                               include_preprocessing=False, weights='imagenet')
model_regnety004.trainable = False

regnety004 = keras.Sequential([
    model_regnety004,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

regnety004.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnety004 = regnety004.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 81s 218ms/step - loss: 2.1550 - accuracy: 0.2495 - f1_score: 0.2030 - precision: 0.2663 - recall: 0.1875 - val_loss: 2.0522 - val_accuracy: 0.3313 - val_f1_score: 0.3225 - val_precision: 0.2946 - val_recall: 0.3578
Epoch 2/10
329/329 [==============================] - 69s 209ms/step - loss: 1.9815 - accuracy: 0.3444 - f1_score: 0.3505 - precision: 0.2865 - recall: 0.4587 - val_loss: 1.9534 - val_accuracy: 0.3596 - val_f1_score: 0.3534 - val_precision: 0.2670 - val_recall: 0.5241
Epoch 3/10
329/329 [==============================] - 69s 209ms/step - loss: 1.8993 - accuracy: 0.3770 - f1_score: 0.3709 - precision: 0.2755 - recall: 0.5707 - val_loss: 1.8959 - val_accuracy: 0.3822 - val_f1_score: 0.3601 - val_precision: 0.2583 - val_recall: 0.5959
Epoch 4/10
329/329 [==============================] - 69s 210ms/step - loss: 1.8483 - accuracy: 0.3961 - f1_score: 0.3753 - precision: 0.2685 - recall: 0.6254 - val_loss: 1.8569 - val_accuracy:

In [14]:
model_regnety004.trainable = True
fine_tune_at = int(len(model_regnety004.layers)*0.7)
for layer in model_regnety004.layers[:fine_tune_at]:
    layer.trainble = False
    
regnety004.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnety004_fine = regnety004.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_regnety004.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 3379s 10s/step - loss: 1.4768 - accuracy: 0.4808 - f1_score: 0.3529 - precision: 0.2332 - recall: 0.7636 - val_loss: 0.9131 - val_accuracy: 0.6764 - val_f1_score: 0.3883 - val_precision: 0.2430 - val_recall: 0.9668
Epoch 11/25
329/329 [==============================] - 3372s 10s/step - loss: 0.6933 - accuracy: 0.7605 - f1_score: 0.3904 - precision: 0.2438 - recall: 0.9798 - val_loss: 0.5845 - val_accuracy: 0.7993 - val_f1_score: 0.3928 - val_precision: 0.2452 - val_recall: 0.9867
Epoch 12/25
329/329 [==============================] - 3377s 10s/step - loss: 0.4559 - accuracy: 0.8438 - f1_score: 0.3922 - precision: 0.2444 - recall: 0.9925 - val_loss: 0.4690 - val_accuracy: 0.8374 - val_f1_score: 0.3904 - val_precision: 0.2430 - val_recall: 0.9934
Epoch 13/25
329/329 [==============================] - 3399s 10s/step - loss: 0.3396 - accuracy: 0.8862 - f1_score: 0.3915 - precision: 0.2437 - recall: 0.9955 - val_loss: 0.4024 - val_accur

In [15]:
metric_calc(regnety004)

95/95 - 18s - 18s/epoch - 192ms/step


(0.9023217247097844,
 0.9024547059342818,
 0.9023217247097844,
 0.9020754044409172)

In [16]:
test_loss, test_acc, test_f1, test_prec, test_rec = regnety004.evaluate(test_batches, verbose=2)

95/95 - 16s - loss: 0.2907 - accuracy: 0.9023 - f1_score: 0.3912 - precision: 0.2434 - recall: 0.9964 - 16s/epoch - 172ms/step


In [17]:
regnety004.save("models/regnety004_weights")

INFO:tensorflow:Assets written to: models/regnety004_weights\assets


INFO:tensorflow:Assets written to: models/regnety004_weights\assets


# RegNetY006

In [12]:
model_regnety006 = tf.keras.applications.regnet.RegNetY006(input_shape=input_shape+(3,), include_top=False, 
                                               include_preprocessing=False, weights='imagenet')
model_regnety006.trainable = False

regnety006 = keras.Sequential([
    model_regnety006,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

regnety006.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnety006 = regnety006.fit(train_batches,
                    epochs=epoch,
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 65s 183ms/step - loss: 2.1120 - accuracy: 0.2365 - f1_score: 0.2652 - precision: 0.1888 - recall: 0.4555 - val_loss: 2.0201 - val_accuracy: 0.2818 - val_f1_score: 0.2993 - val_precision: 0.1970 - val_recall: 0.6234
Epoch 2/10
329/329 [==============================] - 58s 176ms/step - loss: 1.9135 - accuracy: 0.3283 - f1_score: 0.3271 - precision: 0.2224 - recall: 0.6203 - val_loss: 1.8741 - val_accuracy: 0.3542 - val_f1_score: 0.3312 - val_precision: 0.2188 - val_recall: 0.6809
Epoch 3/10
329/329 [==============================] - 58s 177ms/step - loss: 1.8207 - accuracy: 0.3719 - f1_score: 0.3484 - precision: 0.2338 - recall: 0.6847 - val_loss: 1.7870 - val_accuracy: 0.3953 - val_f1_score: 0.3505 - val_precision: 0.2305 - val_recall: 0.7318
Epoch 4/10
329/329 [==============================] - 58s 176ms/step - loss: 1.7629 - accuracy: 0.4000 - f1_score: 0.3590 - precision: 0.2392 - recall: 0.7210 - val_loss: 1.7314 - val_accuracy:

In [13]:
model_regnety006.trainable = True
fine_tune_at = int(len(model_regnety006.layers)*0.7)
for layer in model_regnety006.layers[:fine_tune_at]:
    layer.trainble = False
    
regnety006.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_regnety006_fine = regnety006.fit(train_batches,
                    epochs=total_epochs,
                    initial_epoch=history_regnety006.epoch[-1],
                    validation_data=val_batches, 
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 2457s 7s/step - loss: 0.6766 - accuracy: 0.7722 - f1_score: 0.3861 - precision: 0.2416 - recall: 0.9630 - val_loss: 0.3222 - val_accuracy: 0.8832 - val_f1_score: 0.4014 - val_precision: 0.2514 - val_recall: 0.9957
Epoch 11/25
329/329 [==============================] - 2442s 7s/step - loss: 0.1967 - accuracy: 0.9332 - f1_score: 0.4154 - precision: 0.2623 - recall: 0.9983 - val_loss: 0.1844 - val_accuracy: 0.9391 - val_f1_score: 0.4099 - val_precision: 0.2579 - val_recall: 0.9993
Epoch 12/25
329/329 [==============================] - 2441s 7s/step - loss: 0.0922 - accuracy: 0.9707 - f1_score: 0.4211 - precision: 0.2668 - recall: 0.9998 - val_loss: 0.1593 - val_accuracy: 0.9492 - val_f1_score: 0.4267 - val_precision: 0.2713 - val_recall: 0.9990
Epoch 13/25
329/329 [==============================] - 2441s 7s/step - loss: 0.0477 - accuracy: 0.9852 - f1_score: 0.4263 - precision: 0.2709 - recall: 0.9999 - val_loss: 0.1328 - val_accuracy:

In [14]:
metric_calc(regnety006)

95/95 - 15s - 15s/epoch - 161ms/step


(0.9754560530679933, 0.975447720879839, 0.9754560530679933, 0.9754276707940625)

In [15]:
test_loss, test_acc, test_f1, test_prec, test_rec = regnety006.evaluate(test_batches, verbose=2)

95/95 - 14s - loss: 0.1010 - accuracy: 0.9755 - f1_score: 0.4440 - precision: 0.2855 - recall: 0.9985 - 14s/epoch - 142ms/step


In [16]:
regnety006.save("models/regnety006_weights")

INFO:tensorflow:Assets written to: models/regnety006_weights\assets


INFO:tensorflow:Assets written to: models/regnety006_weights\assets


# InceptionResNetV2

In [12]:
input_shapeIR = (128,75)

train_msIR,train_labelsIR = preprocess_dataset(train_files, input_shapeIR)
test_msIR,test_labelsIR = preprocess_dataset(test_files, input_shapeIR)
val_msIR,val_labelsIR = preprocess_dataset(val_files, input_shapeIR)

train_datasetIR = tf.data.Dataset.from_tensor_slices((train_msIR, train_labelsIR))
test_datasetIR = tf.data.Dataset.from_tensor_slices((test_msIR, test_labelsIR))
val_datasetIR = tf.data.Dataset.from_tensor_slices((val_msIR, val_labelsIR))

train_batchesIR = train_datasetIR.batch(batch_size)
test_batchesIR = test_datasetIR.batch(batch_size)
val_batchesIR = val_datasetIR.batch(batch_size)

In [13]:
model_inceptionresnetv2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=input_shapeIR+(3,), 
                                                                                      include_top=False, weights='imagenet')
model_inceptionresnetv2.trainable = False

inceptionresnetv2 = keras.Sequential([
    model_inceptionresnetv2,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                  
])

inceptionresnetv2.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_inceptionresnetv2 = inceptionresnetv2.fit(train_batchesIR,
                    epochs=epoch,
                    validation_data=val_batchesIR,
                    verbose=1,
                    shuffle=True)

219055592/219055592 [==============================] - 190s 1us/step
Epoch 1/10
329/329 [==============================] - 267s 777ms/step - loss: 1.4992 - accuracy: 0.4968 - f1_score: 0.3440 - precision: 0.2154 - recall: 0.8587 - val_loss: 1.3044 - val_accuracy: 0.5465 - val_f1_score: 0.3549 - val_precision: 0.2194 - val_recall: 0.9285
Epoch 2/10
329/329 [==============================] - 253s 769ms/step - loss: 1.0791 - accuracy: 0.6324 - f1_score: 0.3641 - precision: 0.2256 - recall: 0.9443 - val_loss: 1.0697 - val_accuracy: 0.6384 - val_f1_score: 0.3600 - val_precision: 0.2218 - val_recall: 0.9565
Epoch 3/10
329/329 [==============================] - 254s 771ms/step - loss: 0.9677 - accuracy: 0.6729 - f1_score: 0.3672 - precision: 0.2271 - recall: 0.9587 - val_loss: 1.0042 - val_accuracy: 0.6572 - val_f1_score: 0.3655 - val_precision: 0.2256 - val_recall: 0.9623
Epoch 4/10
329/329 [==============================] - 253s 771ms/step - loss: 0.9057 - accuracy: 0.6957 - f1_score: 0.368

In [14]:
model_inceptionresnetv2.trainable = True
fine_tune_at = int(len(model_inceptionresnetv2.layers)*0.7)
for layer in model_inceptionresnetv2.layers[:fine_tune_at]:
    layer.trainble = False
    
inceptionresnetv2.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_inceptionresnetv2_fine = inceptionresnetv2.fit(train_batchesIR,
                    epochs=total_epochs,
                    initial_epoch=history_inceptionresnetv2.epoch[-1],
                    validation_data=val_batchesIR,
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 1661s 5s/step - loss: 0.5296 - accuracy: 0.8346 - f1_score: 0.3713 - precision: 0.2290 - recall: 0.9817 - val_loss: 0.7143 - val_accuracy: 0.8825 - val_f1_score: 0.3891 - val_precision: 0.2419 - val_recall: 0.9953
Epoch 11/25
329/329 [==============================] - 1832s 6s/step - loss: 0.1403 - accuracy: 0.9545 - f1_score: 0.3770 - precision: 0.2324 - recall: 0.9981 - val_loss: 0.2896 - val_accuracy: 0.8997 - val_f1_score: 0.3821 - val_precision: 0.2363 - val_recall: 0.9987
Epoch 12/25
329/329 [==============================] - 1900s 6s/step - loss: 0.0568 - accuracy: 0.9819 - f1_score: 0.3782 - precision: 0.2333 - recall: 0.9998 - val_loss: 0.1663 - val_accuracy: 0.9535 - val_f1_score: 0.3788 - val_precision: 0.2338 - val_recall: 0.9987
Epoch 13/25
329/329 [==============================] - 1867s 6s/step - loss: 0.0297 - accuracy: 0.9902 - f1_score: 0.3790 - precision: 0.2339 - recall: 0.9998 - val_loss: 0.1299 - val_accuracy:

In [18]:
metric_calc(inceptionresnetv2)

95/95 - 87s - 87s/epoch - 912ms/step


(0.9723051409618574,
 0.9723409226434793,
 0.9723051409618574,
 0.9722750884300165)

In [19]:
test_loss, test_acc, test_f1, test_prec, test_rec = inceptionresnetv2.evaluate(test_batchesIR, verbose=2)

95/95 - 92s - loss: 0.1323 - accuracy: 0.9723 - f1_score: 0.3858 - precision: 0.2390 - recall: 0.9995 - 92s/epoch - 964ms/step


In [20]:
inceptionresnetv2.save("models/inceptionresnetv2_weights")

INFO:tensorflow:Assets written to: models/inceptionresnetv2_weights\assets


INFO:tensorflow:Assets written to: models/inceptionresnetv2_weights\assets


# Xception

In [17]:
input_shapeX = (128,71)

train_msX,train_labelsX = preprocess_dataset(train_files, input_shapeX)
test_msX,test_labelsX = preprocess_dataset(test_files, input_shapeX)
val_msX,val_labelsX = preprocess_dataset(val_files, input_shapeX)

train_datasetX = tf.data.Dataset.from_tensor_slices((train_msX, train_labelsX))
test_datasetX = tf.data.Dataset.from_tensor_slices((test_msX, test_labelsX))
val_datasetX = tf.data.Dataset.from_tensor_slices((val_msX, val_labelsX))

train_batchesX = train_datasetX.batch(batch_size)
test_batchesX = test_datasetX.batch(batch_size)
val_batchesX = val_datasetX.batch(batch_size)

In [18]:
model_xception = tf.keras.applications.xception.Xception(input_shape=input_shapeX+(3,), include_top=False, weights='imagenet')
model_xception.trainable = False

xception = keras.Sequential([
    model_xception,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labels))                
])

xception.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_xception = xception.fit(train_batchesX,
                    epochs=epoch,
                    validation_data=val_batchesX,
                    verbose=1,
                    shuffle=True)

Epoch 1/10
329/329 [==============================] - 206s 615ms/step - loss: 1.4743 - accuracy: 0.4998 - f1_score: 0.3737 - precision: 0.2485 - recall: 0.7741 - val_loss: 1.2242 - val_accuracy: 0.6104 - val_f1_score: 0.3966 - val_precision: 0.2554 - val_recall: 0.8878
Epoch 2/10
329/329 [==============================] - 202s 615ms/step - loss: 1.1422 - accuracy: 0.6279 - f1_score: 0.3947 - precision: 0.2522 - recall: 0.9093 - val_loss: 1.1060 - val_accuracy: 0.6485 - val_f1_score: 0.3942 - val_precision: 0.2514 - val_recall: 0.9134
Epoch 3/10
329/329 [==============================] - 202s 614ms/step - loss: 1.0475 - accuracy: 0.6649 - f1_score: 0.3946 - precision: 0.2506 - recall: 0.9291 - val_loss: 1.0489 - val_accuracy: 0.6616 - val_f1_score: 0.3937 - val_precision: 0.2501 - val_recall: 0.9257
Epoch 4/10
329/329 [==============================] - 202s 614ms/step - loss: 0.9931 - accuracy: 0.6842 - f1_score: 0.3941 - precision: 0.2495 - recall: 0.9387 - val_loss: 1.0160 - val_accur

In [19]:
model_xception.trainable = True
fine_tune_at = int(len(model_xception.layers)*0.7)
for layer in model_xception.layers[:fine_tune_at]:
    layer.trainble = False
    
xception.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/fine_tune_epochs),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_score, precision, recall])

history_xception_fine = xception.fit(train_batchesX,
                    epochs=total_epochs,
                    initial_epoch=history_xception.epoch[-1],
                    validation_data=val_batchesX,
                    verbose=1,
                    shuffle=True)

Epoch 10/25
329/329 [==============================] - 1098s 3s/step - loss: 0.5198 - accuracy: 0.8158 - f1_score: 0.3960 - precision: 0.2536 - recall: 0.9488 - val_loss: 0.2938 - val_accuracy: 0.9071 - val_f1_score: 0.4168 - val_precision: 0.2636 - val_recall: 0.9963
Epoch 11/25
329/329 [==============================] - 1088s 3s/step - loss: 0.0742 - accuracy: 0.9764 - f1_score: 0.4247 - precision: 0.2697 - recall: 0.9997 - val_loss: 0.1211 - val_accuracy: 0.9650 - val_f1_score: 0.4341 - val_precision: 0.2774 - val_recall: 0.9990
Epoch 12/25
329/329 [==============================] - 1077s 3s/step - loss: 0.0144 - accuracy: 0.9965 - f1_score: 0.4301 - precision: 0.2740 - recall: 1.0000 - val_loss: 0.1118 - val_accuracy: 0.9704 - val_f1_score: 0.4354 - val_precision: 0.2784 - val_recall: 0.9993
Epoch 13/25
329/329 [==============================] - 1060s 3s/step - loss: 0.0030 - accuracy: 0.9997 - f1_score: 0.4342 - precision: 0.2774 - recall: 1.0000 - val_loss: 0.1029 - val_accuracy:

In [23]:
metric_calc(xception)

95/95 - 47s - 47s/epoch - 494ms/step


(0.9733001658374792,
 0.9733766505244987,
 0.9733001658374792,
 0.9732853701001281)

In [20]:
test_loss, test_acc, test_f1, test_prec, test_rec = xception.evaluate(test_batchesX, verbose=2)

95/95 - 52s - loss: 0.1100 - accuracy: 0.9733 - f1_score: 0.4448 - precision: 0.2862 - recall: 0.9987 - 52s/epoch - 552ms/step


In [21]:
xception.save("models/xception_weights")

INFO:tensorflow:Assets written to: models/xception_weights\assets


INFO:tensorflow:Assets written to: models/xception_weights\assets
